In [2]:
import os
import re
import math
from ast import literal_eval
from collections import Counter
from glob import glob
import warnings
import jenkspy


import pandas as pd
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import seaborn as sns

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, _tree
from sklearn.metrics import f1_score, classification_report
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import (
    cross_val_score,
    ShuffleSplit,
    GridSearchCV,
    train_test_split
)
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)


In [3]:
df1 = pd.read_csv("./Datafiles/dearr0.1_0.06.csv")

df1.fillna(0,inplace=True)

df_raw1 = df1[['File','Completeness', 'Conciseness', 'cor.mean', 'cov.mean', 'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 
 'kurtosis.mean', 'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr', 'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean', 'sparsity.mean',
 't_mean.mean', 'var.mean', 'ClassImbRatio', 'ClassOverlapPerc', 'OutlierPerc', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin', 'nr_inst', 'nr_num','attr_conc.mean', 
 'attr_ent.mean', 'LabelIssuesPerc','nUnique', 'ena', 'snr.mean', 'cEntropy', 'FeatureAlgo']]



In [4]:
def calcu_scores(df):
    scores = []
    labels = df['FeatureAlgo']
    features = df.drop(['FeatureAlgo','File'], axis=1)

    le = LabelEncoder()
    features = features.apply(le.fit_transform)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=42, train_size=0.8)

    param_grid_mlp = {
        'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
        'solver': ['sgd', 'adam'],
        'alpha': [0.0001, 0.05],
        'max_iter': [100, 200, 500]
    }

    param_grid_lr = {
        'penalty': ['l1', 'l2'],
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['saga']
    }

    param_grid_svc = {
        'C': [0.01, 0.1, 1, 10, 100],
        'max_iter': [1000, 2000, 3000]
    }

    param_grid_dt = {'max_features': ['auto', 'sqrt', 'log2'],
              'ccp_alpha': [0.1, .01, .001],
              'max_depth' : [5, 7, 9],
              'criterion' :['gini', 'entropy']
             }

    mlp_clf = GridSearchCV(MLPClassifier(random_state=1), param_grid_mlp, cv=5)
    lr_clf = GridSearchCV(LogisticRegression(random_state=0, max_iter=1000), param_grid_lr, cv=5)
    dt_clf = GridSearchCV(DecisionTreeClassifier(), param_grid_dt, cv=5)
    svc_clf = GridSearchCV(LinearSVC(random_state=0), param_grid_svc, cv=5)

    mlp_clf.fit(X_train, y_train)
    lr_clf.fit(X_train, y_train)
    dt_clf.fit(X_train, y_train)
    svc_clf.fit(X_train, y_train)

    mlp_predicted = mlp_clf.predict(X_test)
    lr_predicted = lr_clf.predict(X_test)
    dt_predicted = dt_clf.predict(X_test)
    svc_predicted = svc_clf.predict(X_test)

    for clf_name, predicted in zip(
        ['MLP', 'LR', 'RF', 'SVC'],
        [mlp_predicted, lr_predicted, dt_predicted, svc_predicted]
    ):
        scores.append(f1_score(predicted, y_test, average='macro'))
        scores.append(f1_score(predicted, y_test, average='micro'))
        scores.append(f1_score(predicted, y_test, average='weighted'))

    return scores

In [5]:
def get_rules_with_support_confidence(tree_model, feature_names, X, y, label_encoder):
    tree_ = tree_model.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    rules = []

    def recurse(node, path, conds):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            left_path = path.copy()
            left_path.append(f"({name} <= {threshold:.3f})")
            recurse(tree_.children_left[node], left_path, conds)

            right_path = path.copy()
            right_path.append(f"({name} > {threshold:.3f})")
            recurse(tree_.children_right[node], right_path, conds)
        else:
            path_str = " and ".join(path)
            support = tree_.n_node_samples[node] / tree_.n_node_samples[0]
            value = tree_.value[node][0]
            predicted_class_index = np.argmax(value)
            predicted_class = label_encoder.inverse_transform([predicted_class_index])[0]
            confidence = value[predicted_class_index] / np.sum(value)
            rules.append((path_str, predicted_class, support, confidence))

    recurse(0, [], rules)

    return pd.DataFrame(rules, columns=["Rule", "Predicted_Class", "Support", "Confidence"])

In [5]:
res0 = calcu_scores(df_raw1)
print(res0)

/home/d19125691/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/d19125691/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/d19125691/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/d19125691/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


[0.36888888888888893, 0.6571428571428571, 0.6895238095238095, 0.27813953488372095, 0.6571428571428571, 0.7362126245847177, 0.21548821548821548, 0.45714285714285713, 0.4956228956228957, 0.42349549549549553, 0.6857142857142857, 0.7334877734877735]


In [6]:
print("Average accuracy of weighted classifications",(res0[2]+res0[5]+res0[8]+res0[11])/4)


Average accuracy of weighted classifications 0.6637117758047991


In [7]:
le = LabelEncoder()

X = df_raw1.drop(['File', 'FeatureAlgo'], axis=1)
labels = df_raw1['FeatureAlgo']

le = LabelEncoder()
features = le.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(X, features, random_state=42, train_size=0.8)

clf = DecisionTreeClassifier(max_depth=40, random_state=0)
clf.fit(X_train, y_train)  

rule_df1 = get_rules_with_support_confidence(clf, feature_names=X_train.columns.tolist(), X=X_train, y=y_train, label_encoder=le)

print(rule_df1.sort_values(by='Support', ascending=False).head(10))

                                                 Rule Predicted_Class  \
32  (nr_inst > 347.000) and (range.mean > 0.749) a...       chisquare   
12  (nr_inst <= 347.000) and (cEntropy > 0.416) an...              GR   
31  (nr_inst > 347.000) and (range.mean > 0.749) a...       chisquare   
1   (nr_inst <= 347.000) and (cEntropy > 0.416) an...       chisquare   
0        (nr_inst <= 347.000) and (cEntropy <= 0.416)       chisquare   
18  (nr_inst <= 347.000) and (cEntropy > 0.416) an...              GR   
36  (nr_inst > 347.000) and (range.mean > 0.749) a...       chisquare   
30  (nr_inst > 347.000) and (range.mean > 0.749) a...            fcbf   
25  (nr_inst > 347.000) and (range.mean > 0.749) a...       chisquare   
24  (nr_inst > 347.000) and (range.mean > 0.749) a...       chisquare   

     Support  Confidence  
32  0.246377    1.000000  
12  0.152174    1.000000  
31  0.079710    1.000000  
1   0.050725    1.000000  
0   0.036232    1.000000  
18  0.036232    1.000000  
36  0.0

In [8]:
rule_df1.to_csv("rulesfromraw.csv",index=False)

We get rules with high support and confidence only for Chisquare and GR. If encoded in ontology we get only two feature selection techniques as recommendation. The main reason for this is class imbalance. Hence, random sampling and smote is applied. 

In [6]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from collections import Counter

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [7]:
def train_model_with_label_names(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)

    X = df.drop(['FeatureAlgo'], axis=1)
    y = df['FeatureAlgo']
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    ros = RandomOverSampler(random_state=33)
    smote = SMOTE(random_state=33)
    pipeline = Pipeline([('ros', ros), ('smote', smote)])

    X_resampled, y_resampled = pipeline.fit_resample(X, y_encoded)
    print(X_resampled.shape)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)

    tree_classifier = DecisionTreeClassifier(random_state=random_state)
    cross_val_scores = cross_val_score(tree_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores

    tree_classifier.fit(X_train, y_train)

    y_type_pred = tree_classifier.predict(X_test)

    print("Feature_Algo Report (all features):")
    print(classification_report(label_encoder.inverse_transform(y_test), label_encoder.inverse_transform(y_type_pred)))
    return tree_classifier, label_encoder, X_train, y_train

X = df_raw1.drop(['File'], axis=1)

tree_classifier,label_encoder,X_train, y_train = train_model_with_label_names(X)

rule_df2 = get_rules_with_support_confidence(tree_classifier,
 feature_names=X_train.columns.tolist(), X=X_train, y=y_train, label_encoder=label_encoder)

print(rule_df2.sort_values(by='Support', ascending=False).head(10))

(460, 39)
Feature_Algo Report (all features):
              precision    recall  f1-score   support

          GR       0.78      0.93      0.85        15
          MI       1.00      1.00      1.00        16
   chisquare       0.92      0.63      0.75        19
        fcbf       0.91      1.00      0.95        21
      relief       0.95      1.00      0.98        21

    accuracy                           0.91        92
   macro avg       0.91      0.91      0.91        92
weighted avg       0.92      0.91      0.91        92

                                                 Rule Predicted_Class  \
7   (nr_inst <= 167.000) and (eigenvalues.mean > 3...              MI   
39  (nr_inst > 167.000) and (nUnique > 6.500) and ...          relief   
15  (nr_inst > 167.000) and (nUnique <= 6.500) and...              GR   
35  (nr_inst > 167.000) and (nUnique <= 6.500) and...            fcbf   
24  (nr_inst > 167.000) and (nUnique <= 6.500) and...       chisquare   
12  (nr_inst > 167.000) and

In [14]:
rule_df2.to_csv("rulesfromrandomsmote.csv",index=False)

In [15]:
print(df_raw1['FeatureAlgo'].value_counts())

chisquare    92
GR           51
fcbf         16
relief        8
MI            6
Name: FeatureAlgo, dtype: int64


In [16]:
def train_model_with_label_names(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)

    X = df.drop(['FeatureAlgo'], axis=1)
    y = df['FeatureAlgo']
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    smote = SMOTE(random_state=33)
   # pipeline = Pipeline([('ros', ros), ('smote', smote)])

    X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)

    tree_classifier = DecisionTreeClassifier(random_state=random_state)
    cross_val_scores = cross_val_score(tree_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores

    tree_classifier.fit(X_train, y_train)

    y_type_pred = tree_classifier.predict(X_test)

    print("Feature_Algo Report (all features):")
    print(classification_report(label_encoder.inverse_transform(y_test), label_encoder.inverse_transform(y_type_pred)))
    return tree_classifier, label_encoder, X_train, y_train

X = df_raw1.drop(['File'], axis=1)

tree_classifier,label_encoder,X_train, y_train = train_model_with_label_names(X)

rule_df3 = get_rules_with_support_confidence(tree_classifier,
 feature_names=X_train.columns.tolist(), X=X_train, y=y_train, label_encoder=label_encoder)

print(rule_df3.sort_values(by='Support', ascending=False).head(10))

Feature_Algo Report (all features):
              precision    recall  f1-score   support

          GR       0.50      0.67      0.57        15
          MI       0.93      0.88      0.90        16
   chisquare       0.64      0.37      0.47        19
        fcbf       0.73      0.90      0.81        21
      relief       1.00      0.95      0.98        21

    accuracy                           0.76        92
   macro avg       0.76      0.75      0.75        92
weighted avg       0.77      0.76      0.75        92

                                                 Rule Predicted_Class  \
41  (var.mean > 121461.484) and (attr_ent.mean <= ...              MI   
33  (var.mean <= 121461.484) and (nUnique > 3.500)...          relief   
27  (var.mean <= 121461.484) and (nUnique <= 3.500...            fcbf   
0   (var.mean <= 121461.484) and (nUnique <= 3.500...              GR   
18  (var.mean <= 121461.484) and (nUnique <= 3.500...       chisquare   
48  (var.mean > 121461.484) and (attr

In [17]:
rule_df3.sort_values(by='Support', ascending=False)

,Rule,Predicted_Class,Support,Confidence
41,(var.mean > 121461.484) and (attr_ent.mean <= ...,MI,0.165761,1.0
33,(var.mean <= 121461.484) and (nUnique > 3.500)...,relief,0.138587,1.0
27,(var.mean <= 121461.484) and (nUnique <= 3.500...,fcbf,0.127717,1.0
0,(var.mean <= 121461.484) and (nUnique <= 3.500...,GR,0.084239,1.0
18,(var.mean <= 121461.484) and (nUnique <= 3.500...,chisquare,0.070652,1.0
48,(var.mean > 121461.484) and (attr_ent.mean > 1...,chisquare,0.038043,1.0
34,(var.mean <= 121461.484) and (nUnique > 3.500)...,MI,0.024457,1.0
4,(var.mean <= 121461.484) and (nUnique <= 3.500...,GR,0.021739,1.0
6,(var.mean <= 121461.484) and (nUnique <= 3.500...,relief,0.021739,1.0
7,(var.mean <= 121461.484) and (nUnique <= 3.500...,GR,0.021739,1.0


In [18]:
rule_df2.to_csv("rulesfromsmote.csv",index=False)